In [2]:
print(__doc__)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding, LSTM
from keras.optimizers import Adam

import itertools
import numpy as np
import pandas as pd

import nltk

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

np.random.seed(7)

Automatically created module for IPython interactive environment


In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
dfSeventh = pd.read_csv('./textsSeventhgrade.csv')
dfEighth = pd.read_csv('./textsEighthgrade.csv')
dfNinth = pd.read_csv('./textsNinthgrade.csv')
dfTenth = pd.read_csv('./textsTenthGrade.csv')
dfEleventh = pd.read_csv('./textsEleventhgrade.csv')

In [ ]:
groupedSeventh = dfSeventh.groupby('id').apply(lambda x: "%s" % ''.join(x['value']))
groupedEighth = dfEighth.groupby('id').apply(lambda x: "%s" % ''.join(x['value']))
groupedNinth = dfNinth.groupby('id').apply(lambda x: "%s" % ''.join(x['value']))
groupedTenth = dfTenth.groupby('id').apply(lambda x: "%s" % ''.join(x['value']))
groupedEleventh = dfEleventh.groupby('id').apply(lambda x: "%s" % ''.join(x['value']))

In [ ]:
len(groupedSeventh.values)

In [ ]:
data = np.concatenate((np.array(groupedSeventh.values),
                       np.array(groupedEighth.values),
                       np.array(groupedNinth.values),
                       np.array(groupedTenth.values),
                       np.array(groupedEleventh.values)  )) 
labels = np.concatenate((np.full(len(groupedSeventh.values), 0),
                         np.full(len(groupedEighth.values), 1),
                         np.full(len(groupedNinth.values), 2),
                         np.full(len(groupedTenth.values), 3),
                         np.full(len(groupedEleventh.values), 4)))

In [37]:
len(data)

207

In [39]:
len(labels)

207

In [10]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

### Load the trained embeddings

In [11]:
EMBEDDINGS_DIMESION = 300

In [12]:
embeddingsIndex = dict()
f = open('./pretrained-word-vectors-for-spanish/SBW-vectors-300-min5.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddingsIndex[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddingsIndex))

Loaded 1000654 word vectors.


### Preprocesing the data

In [13]:
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [14]:
def createEmbedingMatrix(tokenizer, vocabSize):
    embeddingMatrix = np.zeros((vocabSize, 300))
    for word, i in tokenizer.word_index.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            embeddingMatrix[i] = embeddingVector
    return embeddingMatrix

In [15]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    vocabSize = len(tokenizer.word_index) + 1
    encodedData = tokenizer.texts_to_sequences(x)
    maxLength = 500
    paddedData = pad_sequences(encodedData, maxlen=maxLength, padding='post')
    embeddings = createEmbedingMatrix(tokenizer, vocabSize)
    
    return paddedData

In [16]:
def createTrainData(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    vocabSize = len(tokenizer.word_index) + 1
    encodedData = tokenizer.texts_to_sequences(x)
    maxLength = 500
    paddedData = pad_sequences(encodedData, maxlen=maxLength, padding='post')
    embeddings = createEmbedingMatrix(tokenizer, vocabSize)
    
    return paddedData, embeddings, maxLength, vocabSize

In [17]:
x_train_tokenize, x_train_embeddings, x_train_max_features, x_train_vocab_size  = createTrainData(x_train)
x_test_tokenize = tokenize(x_test)

In [18]:
x_train_embeddings.shape

(13424, 300)

In [19]:
len(x_test_tokenize)

42

In [20]:
len(x_train_tokenize)

165

### Creating the model

In [21]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
 
    print(cm)
 
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
 
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [22]:
def testModels(paramsArr):
    resultMetrics = list();
    resultPredict = list();
    
    for params in paramsArr:
        batch_size = params[0]
        numOfCells = params[1]
        epochs = params[2]
        # Model definition
        model = Sequential()
        adam = Adam(lr=0.001)
        embeddingLayer = Embedding(x_train_vocab_size,
                                   EMBEDDINGS_DIMESION,
                                   weights=[x_train_embeddings],
                                   input_length=x_train_max_features,
                                   trainable=False)
        model.add(embeddingLayer)
        model.add(LSTM(numOfCells))
        model.add(Dropout(0.1))
        model.add(Dense(5, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
        # Model Train
        model.fit(x_train_tokenize, y_train, epochs=epochs, batch_size=batch_size)
        # Model Prediction
        y_predict = model.predict(x_test_tokenize)
        y = np.argmax(y_predict, axis=1)
        # Metrics calculation
        f1_result_score = f1_score(y_test, y, average='macro')
        precision_result_score = precision_score(y_test, y, average='macro')
        recall_result_score = recall_score(y_test, y, average='macro')
        accuracy_result_score = accuracy_score(y_test, y)
        resultPredict.append(y)
        resultMetrics.append([f1_result_score, precision_result_score, recall_result_score, accuracy_result_score])

    return resultMetrics, resultPredict

In [23]:
testParams = [
  [22, 100, 10],
  [32, 4, 10],
  [64, 16, 20],
  [32, 16, 20],
  [64, 32, 30],
  [64, 100, 30],
  [32, 32, 30],
  [64, 64, 40],
  [64, 64, 40],
]

In [24]:
testParams2 = [
  [32, 16, 20],
  [32, 32, 20],
  [32, 64, 20],
  [32, 128, 20],
  [32, 256, 20],
  [64, 64, 40],
  [64, 128, 40],
  [64, 256, 40],
]

In [3]:
metrics, predictions = testModels(testParams2)

In [53]:
CLASSES = ["Seventh Grade", "Eighth Grade", "Ninth grade", "Tenth grade", "Eleven grade"]

In [54]:
confusionMatrixs = list()

for prediction in predictions:
    confusionMatrix = confusion_matrix(y_test, prediction)
    np.set_printoptions(precision=2)
    confusionMatrixs.append(confusionMatrix)

In [55]:
len(confusionMatrixs)

8

In [35]:
CLASSES = ["Seventh Grade", "Eighth Grade", "Ninth grade", "Tenth grade", "Eleven grade"]

In [ ]:
confusionMatrixs = list()

for prediction in predictions:
    confusionMatrix = confusion_matrix(y_test, prediction)
    np.set_printoptions(precision=2)
    confusionMatrixs.append(confusionMatrix)

len(confusionMatrixs)

In [ ]:
metrics